In [43]:
db_uname='power_user'
db_pswd='deepodds_usf'
db_host='ec2-50-112-29-56.us-west-2.compute.amazonaws.com'
db_name='deepodds'

In [44]:
import pandas as pd

In [45]:
from sqlalchemy import create_engine

In [46]:
def create_connection_postgre():
    """
    Function to create SQL connection object
    :return: SQL Connection object
    """
    engine = create_engine('postgresql://{}:{}@{}/{}'.format(
        db_uname, db_pswd, db_host, db_name))
    conn = engine.connect()
    return conn

In [47]:
read_matches = """
select * from base_matches
where season = '2015/2016'
"""

In [48]:
read_teams = '''
select * from base_teams;
'''

In [107]:
read_match_outcome = '''
select match_api_id, home_team_goal, away_team_goal  
from 
base_matches;
'''

In [108]:
conn = create_connection_postgre()

In [109]:
teams = pd.read_sql(read_teams,conn)

In [110]:
bets = pd.read_sql(read_bets,conn)

In [111]:
outcomes = pd.read_sql(read_match_outcome,conn)

In [112]:
active_bets = bets[bets.is_active].sort_values(by = ['attr_1', 'predicted_odds'], ascending=[True, True])

In [113]:
active_bets.head(2)

,pred_id,match_id,home_team_id,away_team_id,prediction,prediction_type,predicted_odds,bookmaker,bookmaker_odds,is_active,attr_1,attr_2,attr_3,attr_4,attr_5,created_time,update_time
5082,10000,2030271,9910,9906,away,result,1.902890,betway,1.91,True,Week 1,0.5255163301670845,0.007109561405910014,None,None,2018-05-03 19:05:27.863707,2018-05-03 19:05:27.863707
6273,11190,2030269,8302,8315,home,result,1.991429,bet365,2.10,True,Week 1,0.5021519463775299,0.10857089728738001,None,None,2018-05-03 19:07:03.536258,2018-05-03 19:07:03.536258


In [114]:
active_bets.columns

Index(['pred_id', 'match_id', 'home_team_id', 'away_team_id', 'prediction',
       'prediction_type', 'predicted_odds', 'bookmaker', 'bookmaker_odds',
       'is_active', 'attr_1', 'attr_2', 'attr_3', 'attr_4', 'attr_5',
       'created_time', 'update_time'],
      dtype='object')

In [128]:
usecols = ['match_id',
     'home_team_id', 'away_team_id', 'prediction', 
    'predicted_odds', 'bookmaker', 'bookmaker_odds',
    'attr_1'
]

In [129]:
teamid2name = dict(zip(teams.team_api_id, teams.team_long_name))

In [130]:
summary = active_bets[usecols]

In [131]:
summary.home_team_id = summary.home_team_id.map(teamid2name)
summary.away_team_id = summary.away_team_id.map(teamid2name)

/home/kerem/anaconda3/envs/fastai/lib/python3.6/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [132]:
summary.reset_index(inplace=True, drop=True)

In [133]:
summary.leverage = summary.bookmaker_odds / summary.predicted_odds

/home/kerem/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


In [134]:
summary['confidence'] = 1/summary.predicted_odds

/home/kerem/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


### Many Ways of Looking At this

**Environment Variables:**

- Bookmaker Odds
- Predicted Confidence on Outcome
- Multiple Games (Playing Multiple Games in same website to compound odds)
- Multiple Companies (Playing Different Website)

**Reward:**

- Profit/Loss for each action (bet)

**On Going Research:** Reinforcement Learning for training an agent to create bets.

In [144]:
outcome_list = []
for h, a in list(zip(outcomes.home_team_goal,outcomes.away_team_goal)):
    if h < a:
        outcome_list.append('away')
    elif h == a:
        outcome_list.append('draw')
    else:
        outcome_list.append('home')

In [146]:
outcomes['outcomes'] = outcome_list

In [148]:
outcomes.head()

,match_api_id,home_team_goal,away_team_goal,outcomes
0,1786012,0,3,away
1,1786013,1,1,draw
2,1786014,1,1,draw
3,1786015,0,1,away
4,523804,1,0,home


In [152]:
summary = summary.merge(outcomes, left_on='match_id', right_on='match_api_id', how='left').drop(['match_id', 'match_api_id'], 1)

## Strategy 1

Play on highest leverage top n games independently every week, no compound.

Constant bet

In [379]:
summary.columns

Index(['home_team_id', 'away_team_id', 'prediction', 'predicted_odds',
       'bookmaker', 'bookmaker_odds', 'attr_1', 'confidence', 'home_team_goal',
       'away_team_goal', 'outcomes', 'week_id'],
      dtype='object')

In [380]:
summary.head()

,home_team_id,away_team_id,prediction,predicted_odds,bookmaker,bookmaker_odds,attr_1,confidence,home_team_goal,away_team_goal,outcomes,week_id
0,RC Celta de Vigo,Atlético Madrid,away,1.902890,betway,1.91,Week 1,0.525516,0,2,away,1
1,Sevilla FC,Athletic Club de Bilbao,home,1.991429,bet365,2.10,Week 1,0.502152,2,0,home,1
2,Sevilla FC,Athletic Club de Bilbao,home,1.991429,betway,2.10,Week 1,0.502152,2,0,home,1
3,Sevilla FC,Athletic Club de Bilbao,home,1.991429,interwetten,2.00,Week 1,0.502152,2,0,home,1
4,UD Las Palmas,Málaga CF,home,2.383265,bet365,2.70,Week 1,0.419592,1,1,draw,1


In [382]:
summary['leverage_ratio'] = summary.bookmaker_odds / summary.predicted_odds

In [511]:
n = 1

In [505]:
top_conf = summary.groupby('week_id')sort_values(by=['week_id', 'confidence'], ascending=[True, False])

In [506]:
top_lev = summary.groupby('week_id').head(n).sort_values(by=['week_id', 'leverage_ratio'], ascending=[True, False])

In [507]:
top_conf

,home_team_id,away_team_id,prediction,predicted_odds,bookmaker,bookmaker_odds,attr_1,confidence,home_team_goal,away_team_goal,outcomes,week_id,leverage_ratio
0,RC Celta de Vigo,Atlético Madrid,away,1.902890,betway,1.91,Week 1,0.525516,0,2,away,1,1.003736
1,Sevilla FC,Athletic Club de Bilbao,home,1.991429,bet365,2.10,Week 1,0.502152,2,0,home,1,1.054519
915,Tottenham Hotspur,Leicester City,home,1.544810,bet365,1.75,Week 2,0.647329,0,1,away,2,1.132825
916,Tottenham Hotspur,Leicester City,home,1.544810,betway,1.75,Week 2,0.647329,0,1,away,2,1.132825
1006,Athletic Club de Bilbao,SD Eibar,home,1.455951,bet365,1.67,Week 3,0.686836,5,2,home,3,1.147017
1007,Athletic Club de Bilbao,SD Eibar,home,1.455951,betway,1.70,Week 3,0.686836,5,2,home,3,1.167622
1092,FC Barcelona,Atlético Madrid,home,1.407744,bet365,1.50,Week 4,0.710357,2,1,home,4,1.065535
1093,FC Barcelona,Atlético Madrid,home,1.407744,betway,1.53,Week 4,0.710357,2,1,home,4,1.086845
1147,Tottenham Hotspur,Watford,home,1.347511,bet365,1.50,Week 5,0.742109,1,0,home,5,1.113164
1148,Tottenham Hotspur,Watford,home,1.347511,betway,1.50,Week 5,0.742109,1,0,home,5,1.113164


In [508]:
top_lev

,home_team_id,away_team_id,prediction,predicted_odds,bookmaker,bookmaker_odds,attr_1,confidence,home_team_goal,away_team_goal,outcomes,week_id,leverage_ratio
1,Sevilla FC,Athletic Club de Bilbao,home,1.991429,bet365,2.10,Week 1,0.502152,2,0,home,1,1.054519
0,RC Celta de Vigo,Atlético Madrid,away,1.902890,betway,1.91,Week 1,0.525516,0,2,away,1,1.003736
915,Tottenham Hotspur,Leicester City,home,1.544810,bet365,1.75,Week 2,0.647329,0,1,away,2,1.132825
916,Tottenham Hotspur,Leicester City,home,1.544810,betway,1.75,Week 2,0.647329,0,1,away,2,1.132825
1007,Athletic Club de Bilbao,SD Eibar,home,1.455951,betway,1.70,Week 3,0.686836,5,2,home,3,1.167622
1006,Athletic Club de Bilbao,SD Eibar,home,1.455951,bet365,1.67,Week 3,0.686836,5,2,home,3,1.147017
1093,FC Barcelona,Atlético Madrid,home,1.407744,betway,1.53,Week 4,0.710357,2,1,home,4,1.086845
1092,FC Barcelona,Atlético Madrid,home,1.407744,bet365,1.50,Week 4,0.710357,2,1,home,4,1.065535
1147,Tottenham Hotspur,Watford,home,1.347511,bet365,1.50,Week 5,0.742109,1,0,home,5,1.113164
1148,Tottenham Hotspur,Watford,home,1.347511,betway,1.50,Week 5,0.742109,1,0,home,5,1.113164


### Bet on top n games independently

In [346]:
import numpy as np

In [544]:
def simple_strategy(topn, total_investment_start):
    """
    This strategy only takes account playing top n 
    games every week with a fixed amount of money.
    This strategy will not keep into account neither
    compound odds nor cumulative betting.
    
    compound odds: multiplying odds by playing them together,
    if a single match is lost you lose all money.
    
    cumulative betting: continue betting with the total money you have.
    
    Player will start with X dollars will take the profit for each week and
    put it in a profit bin. 
    
    Money will be distributed evenly to each selected game.
    
    topn : sorted data by weeks and selection: leverage_ratio or confidence 
    total_investment_start : Money that will be distributed among bets for the week
    """
    unique_weeks = topn.week_id.unique()
    total_investment = total_investment_start
    week_money = total_investment_start
    total_net_profit = 0
    
    for week in unique_weeks:
        print(f"Week {week}")
        # pick this week's games
        week_games = topn[topn.week_id == week]
        # number of games in the week
        n_games = len(week_games)
        #print(n_games)
        # distribute money evenly, a constant scalar for this strategy
        money_per_game = week_money / n_games
        # matching predictions
        pos = (week_games['prediction'] == week_games['outcomes']).values
        # matching outcomes
        pos_odds = week_games['bookmaker_odds'][pos].values
        #print(pos_odds)
        # calculate this week's profit
        profit_vec = pos_odds*money_per_game
        #print(money_per_game)
        #print(profit_vec)
        week_net_profit = sum(profit_vec) - week_money
        total_net_profit += week_net_profit
        # if no profit made
        # add another investment
        if total_net_profit < 0:
            total_investment += total_investment_start
        #print(f"Week {week} Profit {week_net_profit}")
        #print()
        
    # calculate ROI
    roi = total_net_profit / total_investment
    
    return total_investment, total_net_profit, roi

In [545]:
# pick the top n games based on either leverage_ratio or confidence
n = 1
selection = 'confidence'
topn = summary.sort_values(by=['week_id', selection], ascending=[True, False]).groupby('week_id').head(n)

In [546]:
simple_strategy(topn, 10)

Week 1
Week 2
Week 3
Week 4
Week 5
Week 6
Week 7
Week 8
Week 9
Week 10
Week 11
Week 13
Week 14
Week 15
Week 16
Week 17
Week 18
Week 19
Week 20
Week 53


(20, 55.39999999999999, 2.7699999999999996)

In [540]:
from itertools import product

In [541]:
n_list = []
method_list = []
total_investment_list = []
total_net_profit_list = []
roi_list = []

for n, selection in list(product(range(1, 10), ['leverage_ratio', 'confidence'])):
    topn = summary.sort_values(by=['week_id', selection], ascending=[True, False]).groupby('week_id').head(n)
    total_investment, total_net_profit, roi = simple_strategy(topn, 10)
    
    
    n_list.append(n)
    method_list.append(selection)
    total_investment_list.append(total_investment)
    total_net_profit_list.append(total_net_profit)
    roi_list.append(roi)
    
    
    print("============================================")
    print(f"Top {n} Games Based on {selection} criteria")
    print("============================================")
    print()
    
    print(f"Total Investment Required: {total_investment}")
    print(f"Total Net Profit: {total_net_profit}")
    print(f"ROI: {roi}")
    print()

Top 1 Games Based on leverage_ratio criteria

Total Investment Required: 170
Total Net Profit: 72.0
ROI: 0.4235294117647059

Top 1 Games Based on confidence criteria

Total Investment Required: 20
Total Net Profit: 55.39999999999999
ROI: 2.7699999999999996

Top 2 Games Based on leverage_ratio criteria

Total Investment Required: 210
Total Net Profit: -55.65
ROI: -0.265

Top 2 Games Based on confidence criteria

Total Investment Required: 10
Total Net Profit: 56.8
ROI: 5.68

Top 3 Games Based on leverage_ratio criteria

Total Investment Required: 170
Total Net Profit: 6.800000000000015
ROI: 0.04000000000000009

Top 3 Games Based on confidence criteria

Total Investment Required: 10
Total Net Profit: 53.03333333333335
ROI: 5.3033333333333355

Top 4 Games Based on leverage_ratio criteria

Total Investment Required: 210
Total Net Profit: -40.7
ROI: -0.19380952380952382

Top 4 Games Based on confidence criteria

Total Investment Required: 10
Total Net Profit: 25.549999999999997
ROI: 2.55499

In [542]:
basic_strategy_df = pd.DataFrame({
"top_n" : n_list ,
"criteria": method_list,
"total_investment" : total_investment_list ,
"total_net_profit" : total_net_profit_list,
"roi" : roi_list 
})[["top_n", "criteria", "total_investment", "total_net_profit", "roi"]]

In [543]:
basic_strategy_df.sort_values(by="roi", ascending=False)

,top_n,criteria,total_investment,total_net_profit,roi
3,2,confidence,10,56.800000,5.680000
5,3,confidence,10,53.033333,5.303333
1,1,confidence,20,55.400000,2.770000
7,4,confidence,10,25.550000,2.555000
0,1,leverage_ratio,170,72.000000,0.423529
9,5,confidence,90,7.560000,0.084000
4,3,leverage_ratio,170,6.800000,0.040000
10,6,leverage_ratio,170,3.050000,0.017941
8,5,leverage_ratio,170,0.300000,0.001765
11,6,confidence,190,-1.100000,-0.005789
